# Initialisation et import des librairies

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy    
!pip install sklearn
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
!pip install GEOparse
import GEOparse
import pandas
from keras.models import Sequential
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from keras.layers import Dense
!pip install h5py pyyaml
import tensorflow as tf
import keras.backend
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess) 

import os
if not os.path.isfile("./gdrive/My Drive/GSE13204_family.soft.gz"):
  !wget -O "./gdrive/My Drive/GSE13204_family.soft.gz" "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE13nnn/GSE13204/soft/GSE13204_family.soft.gz" 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Using TensorFlow backend.


     |████████████████████████████████| 276kB 2.6MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
  Created wheel for GEOparse: filename=GEOparse-2.0.1-cp36-none-any.whl size=28695 sha256=08e0a1674bd05116db63471769b733fd3bff78f9a468494a84545b6f718444c3
  Stored in directory: /root/.cache/pip/wheels/bb/52/4c/44df5b0f3b97bf8172881065941a6b6e1c46408290b02daf7f
Successfully built GEOparse
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


# Traitement des données

In [2]:
# Soft file

gse = GEOparse.get_GEO(filepath="./gdrive/My Drive/GSE13204_family.soft.gz", silent = True)

/usr/local/lib/python3.6/dist-packages/GEOparse/GEOparse.py:104: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return parse_GSE(filepath)


In [3]:
gsms = gse.gsms
print(list(gsms.keys())[:10])

['GSM329407', 'GSM329408', 'GSM329409', 'GSM329410', 'GSM329411', 'GSM329412', 'GSM329413', 'GSM329414', 'GSM329415', 'GSM329416']


In [4]:
print(gsms['GSM329411'].table)
print(gsms['GSM329407'].columns)
print(gsms['GSM329407'].metadata)

               ID_REF    VALUE_DS ABS_CALL  DETECTION P-VALUE     VALUE
0      AFFX-BioB-5_at   5614.9300        P           0.000127       NaN
1      AFFX-BioB-M_at   9707.9000        P           0.000044       NaN
2      AFFX-BioB-3_at   5331.4200        P           0.000044       NaN
3      AFFX-BioC-5_at  17928.6000        P           0.000044       NaN
4      AFFX-BioC-3_at  18722.0000        P           0.000044       NaN
...               ...         ...      ...                ...       ...
54670      1570644_at     57.2892        A           0.870361  0.058415
54671      1570645_at    127.5280        A           0.398926  0.158527
54672      1570650_at    214.8240        A           0.601074  0.247337
54673      1570651_at    488.8230        P           0.046143  0.416758
54674      1570653_at    122.2930        A           0.466064  0.152232

[54675 rows x 5 columns]
                                                         description
ID_REF                                   

In [5]:
# Label
lise = []
alignments = {}
for i in gsms.keys():
    lab = gsms[i].metadata['characteristics_ch1'][1]
    lise.append(lab)
    alignments[i] = lab 
for i in list(set(lise)):
    print(i)

leukemia class: CML
leukemia class: Pro-B-ALL with t(11q23)/MLL
leukemia class: T-ALL
leukemia class: AML with normal karyotype + other abnormalities
leukemia class: c-ALL/Pre-B-ALL with t(9;22)
leukemia class: c-ALL/Pre-B-ALL without t(9;22)
leukemia class: ALL with t(1;19)
leukemia class: AML with t(8;21)
leukemia class: AML with t(15;17)
leukemia class: AML complex aberrant karyotype
leukemia class: AML with inv(16)/t(16;16)
leukemia class: AML with t(11q23)/MLL
leukemia class: CLL
leukemia class: MDS
leukemia class: ALL with t(12;21)
leukemia class: Non-leukemia and healthy bone marrow
leukemia class: mature B-ALL with t(8;14)
leukemia class: ALL with hyperdiploid karyotype


In [6]:
# Assigning CEL files with their labels

pandas.set_option('display.max_rows', 10)
print("Number of patients : "+str(len(gsms.keys())))
gpls = gse.gpls
print(gpls.keys())
print(gpls['GPL570'].columns)
print(gpls['GPL570'].table)
#print(gpls.table)
count_list = []
print("Count of genes :")
df = gsms['GSM329411'].table.dropna()
print(len(df))

equivalents = {
"leukemia class: ALL with t(12;21)":0,  # ALL
"leukemia class: c-ALL/Pre-B-ALL with t(9;22)":0, # ALL
"leukemia class: AML with normal karyotype + other abnormalities":1, # AML
"leukemia class: ALL with t(1;19)":0, # ALL
"leukemia class: CLL":2, # CLL
"leukemia class: AML with t(15;17)":1, # AML
"leukemia class: Non-leukemia and healthy bone marrow":5, # Non-Leukemia
"leukemia class: Pro-B-ALL with t(11q23)/MLL":0, # ALL
"leukemia class: ALL with hyperdiploid karyotype":0, # ALL
"leukemia class: T-ALL":0, # ALL
"leukemia class: AML with t(8;21)":1, # AML
"leukemia class: AML with inv(16)/t(16;16)":1, # AML
"leukemia class: MDS":4, # MDS
"leukemia class: mature B-ALL with t(8;14)":0, # ALL
"leukemia class: c-ALL/Pre-B-ALL without t(9;22)":0, # ALL
"leukemia class: AML with t(11q23)/MLL":1, # AML
"leukemia class: CML":3, # CML
"leukemia class: AML complex aberrant karyotype":1, # AML
}
import numpy

X = []
Y = []
count54 = 0
count14 = 0
all_labels = []
for i in gsms:
  j = gsms[i].table.dropna()
  if len(j["VALUE_DS"]) == 54630: 
    Y.append(numpy.array(equivalents[gsms[i].metadata['characteristics_ch1'][1]]))
    all_labels.append(int(equivalents[gsms[i].metadata['characteristics_ch1'][1]]))
    r = j["VALUE_DS"].to_numpy()
    X.append(r)
    count54 += 1
  else:
    count14 +=1


print("STAGE 1 DATA :"+str(count54))   
print("STAGE 2 DATA :"+str(count14))
 

Number of patients : 3248
dict_keys(['GPL570', 'GPL7473'])
                                                                        description
ID                                Affymetrix Probe Set ID LINK_PRE:"https://www....
GB_ACC                            GenBank Accession Number LINK_PRE:"http://www....
SPOT_ID                                                         identifies controls
Species Scientific Name           The genus and species of the organism represen...
Annotation Date                   The date that the annotations for this probe a...
...                                                                             ...
ENTREZ_GENE_ID                    Entrez Gene Database UID LINK_PRE:"http://www....
RefSeq Transcript ID              References to multiple sequences in RefSeq. Th...
Gene Ontology Biological Process  Gene Ontology Consortium Biological Process de...
Gene Ontology Cellular Component  Gene Ontology Consortium Cellular Component de...
Gene Ontology Mol

# Creation des models et apprentissage



In [7]:
from google.colab import drive
drive.mount('/content/drive')
from keras import optimizers

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
def train_fit_test(model, train_x, train_y, test_x, test_y, classWeight):
    opt = optimizers.SGD(lr=1e-2)
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
    model.fit(train_x, train_y, epochs=100, verbose=2, batch_size=32, class_weight=classWeight)
    y_pred = model.predict(test_x)
    y_pred_formated = numpy.argmax(y_pred, axis=1)
    test_y_formated = numpy.argmax(test_y, axis=1)
    precision = precision_score(test_y_formated, y_pred_formated, average="macro")  
    conf = confusion_matrix(test_y_formated, y_pred_formated)
    print(str(conf))
    return precision, conf

def create_model(num_layers, num_neurons):
    model = Sequential()
    model.add(Dense(64, input_dim=54630))
    for i in range(num_layers):
      model.add(Dense(units=num_neurons, activation='relu'))
    model.add(Dense(units=6, activation='sigmoid'))
    return model

def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    import numpy as np

    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

from sklearn.preprocessing import OneHotEncoder
Y_step = numpy.copy(Y)
Y_step2 = numpy.array(Y_step).reshape(-1, 1)
from sklearn.utils.class_weight import compute_class_weight
classWeight = compute_class_weight('balanced', [0,1,2,3,4,5], all_labels) 
classWeight = dict(enumerate(classWeight))
print("Class weight : "+str(classWeight))
from keras.utils import to_categorical
Y_step2 = to_categorical(Y_step2)
classes_weight = {}
print(" Distribution of classes : "+str(classWeight))
print("Y[:5] : "+str(Y_step2[:5]))
print("Y[0] type : "+str(type(Y[0])))


Y_step2 = numpy.array(Y_step2)
X_step = numpy.array(X)
#X_step = numpy.nan_to_num(X_step)
print("X[0] shape : "+str(X[0].shape))
best_mean = -999999
best_model = []
layers = [50, 100, 1000, 1500, 2000]
train_x, test_x, train_y, test_y = train_test_split(X_step, Y_step2, test_size=0.33, random_state=42)
import pickle
with open('/content/drive/My Drive/train_x.pickle', 'wb') as handle:
    pickle.dump(train_x, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/My Drive/train_y.pickle', 'wb') as handle2:
    pickle.dump(train_y, handle2, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/My Drive/test_x.pickle', 'wb') as handle3:
    pickle.dump(test_x, handle3, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/My Drive/test_y.pickle', 'wb') as handle4:
    pickle.dump(test_y, handle4, protocol=pickle.HIGHEST_PROTOCOL)

from sklearn.utils import compute_class_weight
#train_x = numpy.nan_to_num(train_x)
#test_x = numpy.nan_to_num(test_x)
for x in range(5): # Number of Retrain
  for i in  range(5, 7): # Maximum number of hidden layers
      for j in layers: # Maximum number of neurons per layer
        model = create_model(i, j)
        model.summary()
        print("Training...")
        precision, confusion = train_fit_test(model, train_x, train_y, test_x, test_y, classWeight)
        print("\n")
        print(type(model))
        print("||||| Accuracy : "+str(precision)+" |||||")
        print("\n")
        with open("/content/drive/My Drive/infos.txt",'a+') as fh:
          model.summary(print_fn=lambda x: fh.write(x + '\n'))
          fh.write(str(confusion)+"\n")
          fh.write("\n Precision : "+str(precision)+"\n")
          fh.close()
        try:
          v = open("/content/drive/My Drive/best_mod.txt","r")
          best_mean = float(v.read())
          print("Best actual accuracy : "+str(best_mean))
          v.close()
        except:
          v = open("/content/drive/My Drive/best_mod.txt","w")
          v.write(str(precision))
          v.close()
        if (precision > best_mean):
          print("\n\n")
          print(" BEST MODEL ! ")
          print("Saving... ")
          print("\n\n")
          v = open("/content/drive/My Drive/best_mod.txt","w")
          v.write(str(precision))
          v.close()
          model.save('/content/drive/My Drive/model.h5')
          best_mean = precision
          best_model = model
    

Mounted at /content/drive
Class weight : {0: 0.4657777777777778, 1: 0.6445264452644527, 2: 0.7797619047619048, 3: 4.5964912280701755, 4: 1.6957928802588997, 5: 4.7207207207207205}
 Distribution of classes : {0: 0.4657777777777778, 1: 0.6445264452644527, 2: 0.7797619047619048, 3: 4.5964912280701755, 4: 1.6957928802588997, 5: 4.7207207207207205}
Y[:5] : [[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
Y[0] type : <class 'numpy.ndarray'>
X[0] shape : (54630,)



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                3496384   
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[  0   0 254   0   0   0]
 [  0   0 172   0   0   0]
 [  0   0 158   0   0   0]
 [  0   0  21   0   0   0]
 [  0   0  68   0   0   0]
 [  0   0  19   0   0   0]]


<class 'keras.engine.sequential.Sequential'>
||||| Accuracy : 0.038053949903660886 |||||





 BEST MODEL ! 
Saving... 



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 64)                3496384   
_________________________________________________________________
dense_9 (Dense)              (None, 100)               6500      
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_12 (Dense)             (

KeyboardInterrupt: ignored